# Unfitted Mixed FEM (using $H(div)$ FE spaces)

Our goal is to consider the mixed Poisson problem with Dirichlet boundary conditions in an unfitted setting: <br> 
Find $u: \Omega \rightarrow \mathbb{R}^d$, $p : \Omega \rightarrow \mathbb{R}$ such that
\begin{alignat}{2}
u - \nabla p &= \phantom{-} 0 \quad &&\text{in } \Omega, \\
\operatorname{div} u &= -f \quad &&\text{in } \Omega, \\
p &= \phantom{-} p_D \quad &&\text{on } \partial \Omega.
\end{alignat}

Here $\Omega \subset \mathbb{R}^d$ is a bounded domain that is not parametrized by the computational mesh, but contained in a background domain $\widetilde{\Omega}$ such that $\overline \Omega \subseteq \widetilde{\Omega}$ which is triangulated by a simplicial, shape regular and quasi-uniform triangulation $\widetilde{\mathcal{T}_h}$. <br>
We set 
\begin{align}
\mathcal{T}_h &= \big\{T \in \widetilde{\mathcal{T}_h} : \text{meas}(T \cap \Omega) > 0 \big\}, \\
\mathcal{T}_h^{\text{cut}} &= \big\{T \in \widetilde{\mathcal{T}_h} : \text{meas}(T \cap \partial \Omega) > 0 \big\}, \\
\mathcal{T}_h^{\text{int}} &= \mathcal{T}_h \setminus \mathcal{T}_h^{\text{cut}}.
\end{align}

## The Method

To obtain inf-sup stable unfitted discretizations, one usually add stabilization terms like the ghost penalty stabilization. In the case of mixed problems, however, adding such terms would pollute the mass balance. Instead, we showed that one can consider the following weak formulation: <br>
Find $(u_h,\bar p_h) \in \mathbb{RT}^k(\mathcal{T}_h) \times \mathbb{P}^k(\mathcal{T}_h)$ such that 
\begin{alignat}{2}
a_h(u_h,v_h) + b_h(v_h,\bar p_h) &= (v_h \cdot n, p_D)_{\partial \Omega} \qquad &&\forall v_h \in \mathbb{RT}^k(\mathcal{T}_h), \\
b_h(u_h,q_h) &= - (f_h,q_h)_{L^2(\Omega^{\mathcal{T}})} \qquad &&\forall q_h \in \mathbb{P}^k(\mathcal{T}_h),
\end{alignat}
where 
\begin{align}
a_h(u_h,v_h) &:= (u_h,v_h)_{L^2(\Omega)} + \gamma_u j_h (u_h,v_h), \\
b_h(v_h,p_h) &:= (\operatorname{div} v_h,p_h)_{L^2(\Omega^{\mathcal{T}})}.
\end{align}

The following code imports the basic functionality of netgen, ngsolve and ngsxfem. Afterwards, we set some general parameters for this notebook.

In [ ]:
#basic imports
from netgen.geom2d import SplineGeometry
from ngsolve import *
from ngsolve.internal import *
from xfem import *
from xfem.lsetcurv import *
#from ngsolve.la import EigenValues_Preconditioner #where do i need this?

In [ ]:
#parameters
ngsglobals.msg_level = 2
quad_mesh = False
maxh = 0.2
order = 2
geom_order=order+1
gamma_stab = 0

Now, the unfitted geometry is implemented. We consider a ring geometry that is contained inside the square $[-1,1]^2$ with inner radius $r_1 = 0.25$ and outer radius $r_2 = 0.75$. Then, the geometry is described by the following signed distance function <br> <br>
\begin{equation}
\Phi(x) = \Big\vert \sqrt{x_1^2 + x_2^2} - \frac{1}{2}(r_1+r_2) \Big\vert - \frac{r_2-r_1}{2}.
\end{equation}

In [ ]:
square = SplineGeometry()
square.AddRectangle((-1, -1), (1, 1), bc=1)
ngmesh = square.GenerateMesh(maxh=maxh, quad_dominated=quad_mesh)
mesh = Mesh(ngmesh)

r2 = 3 / 4  # outer radius
r1 = 1 / 4  # inner radius
rc = (r1 + r2) / 2.0
rr = (r2 - r1) / 2.0
r = sqrt(x**2 + y**2)
levelset = IfPos(r - rc, r - rc - rr, rc - r - rr)

#DrawDC(levelset,-1.0,2.0,mesh,"x") #from xfem tutorials, webgui error local...


lsetmeshadap = LevelSetMeshAdaptation(mesh, order=geom_order, threshold=0.1,discontinuous_qn=True)
deformation = lsetmeshadap.CalcDeformation(levelset)

lsetp1 = lsetmeshadap.lset_p1

ci = CutInfo(mesh, lsetp1)
hasneg = ci.GetElementsOfType(HASNEG)
pos = ci.GetElementsOfType(POS)
hasif = ci.GetElementsOfType(IF)
hasany = ci.GetElementsOfType(ANY)

# facets used for stabilization:
ba_facets = GetFacetsWithNeighborTypes(mesh, a=hasneg, b=hasif)

EA = ElementAggregation(mesh)
EA.Update(ci.GetElementsOfType(NEG), hasif)
dX = dCut(lsetp1, NEG, definedonelements=hasneg, deformation=deformation)

cut_facets = GetFacetsWithNeighborTypes(mesh,a=hasif,b=hasif,use_and=True)

<b> to do: take a look at the ngsxfem tutorials to plot this... </b> <br>
<b> could include pictures of the patches, see examples ... </b>

## Solving the problem 

We consider the exact solution $p = \sin(x)$ and derive $u = \nabla p$ and $f = - \Delta p$. Then, we approximate $f$ by $f_h$ with ...

In [ ]:
#exact solutions and other stuff here...
p_exact = sin(x)
coeff_f = - (p_exact.Diff(x).Diff(x) + p_exact.Diff(y).Diff(y)).Compile()
u_exact = CF((p_exact.Diff(x),p_exact.Diff(y)))
Pcoeff_f = GridFunction(L2(mesh,order=order))
Pcoeff_f = coeff_f

In [ ]:
#FESpaces
Shsbase = HDiv(mesh, order=order, dirichlet=[], dgjumps=True, RT=True, discontinuous=False)
Vhbase = L2(mesh, order=order, dirichlet=[], dgjumps=True)
Vh = Restrict(Vhbase, hasneg)
Shs = Restrict(Shsbase, hasneg)
Wh = Shs*Vh

gfw = GridFunction(Wh)
gfu, gfpT = gfw.components[0:2]
(uh,pT), (vh,qT) = Wh.TnT()


h = specialcf.mesh_size
n = Normalize(grad(lsetp1))
nF= specialcf.normal(mesh.dim)

# integration domains:
dxbar = dx(definedonelements=ci.GetElementsOfType(HASNEG), deformation=deformation)
dxinner = dx(definedonelements=ci.GetElementsOfType(NEG), deformation=deformation)
dxcut = dx(definedonelements=ci.GetElementsOfType(IF), deformation=deformation)
ds = dCut(lsetp1, IF, definedonelements=hasif, deformation=deformation)
deb = dx(definedonelements=hasany, deformation=deformation, element_boundary=True)

if gamma_stab > 0:
    #Integration domain for ghost penalty
    dP = dFacetPatch(definedonelements=EA.patch_interior_facets, deformation=deformation)

if gamma_stab > 0:
    facet_restriction = BitArray(mesh.nfacet)
    facet_restriction[:] = EA.patch_interior_facets
    #Draw(BitArrayCF(GetElementsWithNeighborFacets(mesh,facet_restriction)), mesh, "facet_restriction")
    #input("facet_restriction")
    a = RestrictedBilinearForm(Wh, element_restriction=hasneg,
                               facet_restriction=facet_restriction,
                               symmetric=True)
else:
    a = BilinearForm(Wh, symmetric=True,condense=False)

    
# Mixed form diffusion term
a += uh*vh * dX
a += (div(uh) * qT + div(vh) * pT) * dxbar
if gamma_stab > 0:
    a += gamma_stab * (uh - uh.Other()) * (vh - vh.Other()) * dP

# R.h.s. term:
f = LinearForm(Wh)
f += -Pcoeff_f * qT * dxbar
f += p_exact * vh * n * ds


# Assemble system
a.Assemble()
f.Assemble()

rhs = f.vec.CreateVector()
rhs.data = f.vec
# Solve linear system
gfw.vec.data = a.mat.Inverse(Wh.FreeDofs(),inverse="umfpack") * f.vec

p_inner_l2error = sqrt(Integrate((gfpT - p_exact)**2*dxinner, mesh))
ul2error = sqrt(Integrate((gfu - u_exact)**2*dX.order(2*order+3), mesh))
ul2error_bar = sqrt(Integrate((gfu - u_exact)**2*dxbar, mesh))

## Patchwise approximation of $f$

Instead of projecting onto the FE Space, we can also use the ghost penalty stabilization to construct an approximation $f_h$ of $f$. For $\gamma_f > 0$, we find $f_h \in \mathbb{P}^{k_f}(\mathcal{T}_h)$, $k_f \in \mathbb{N}_0$, by solving 
\begin{equation}
(f_h,q_h)_{L^2(\Omega)} + \gamma_f j_h (f_h,q_h) = (f,q_h)_{L^2(\Omega)} \qquad \forall q_h \in \mathbb{P}^{k_f}(\mathcal{T}_h).
\end{equation}

The following code implements this approximation using patches. 

In [ ]:
kf = order
gff = GridFunction(L2(mesh,order=kf))
fh, wh = gff.space.TnT()
lhs_integrals = fh * wh * dX + 0.01*(fh-fh.Other())*(wh-wh.Other()) * dFacetPatch(deformation=deformation)
rhs_integrals = coeff_f * wh * dX
EA2 = ElementAggregation(mesh)
EA2.Update(ci.GetElementsOfType(NEG), ci.GetElementsOfType(IF))
gff.vec.data = PatchwiseSolve(EA2,gff.space,lhs_integrals,rhs_integrals, heapsize=100000000)
Pcoeff_f = gff

## Elementwise Post-processing

By making use of the relation $u=\nabla p$, we can repair the inconsistencies of the approximation of $p$ and achieve higher order convergence. <br>
For each element $T \in \mathcal{T}_h$, we want to find $p_h^\ast \in \mathcal{P}^{k+1}(T)$ such that 
\begin{alignat}{2}
(\nabla p_h^\ast, \nabla q_h^\ast)_T &= (u_h, \nabla q_h^\ast)_T \qquad &&\forall q_h^\ast \in \mathcal{P}^{k+1}(T)\\
(p_h^\ast,1)_T &= (\bar p_h,1)_T \qquad &&\text{if } T \in \mathcal{T}_h^{\text{int}}\\
(p_h^\ast,1)_{T \cap \partial \Omega} &= (p_D,1)_{T \cap \partial \Omega} \qquad &&\text{if } T \in \mathcal{T}_h^{\text{cut}}\\
\end{alignat}

<div align='center'>
    <img src="graphics/elpp.png" width=300px height=300px style="border:none">
</div>

In [ ]:
#FE Spaces for el-wise post-processing
Vhpbase = L2(mesh, order=order+1, dgjumps=False, dirichlet=[]) # order+1
Lhbase = L2(mesh, order=0, dgjumps=False, dirichlet=[])

Vhp = Restrict(Vhpbase, hasneg)
Lh = Restrict(Lhbase, hasneg)
Zh = Vhp * Lh

gflh = GridFunction(Lh)

gfz = GridFunction(Zh)
gfps, gflam = gfz.components

#Test- & Trialfunction
(ps,lam),(vs,mu) = Zh.TnT()

#Bilinear Form
p = RestrictedBilinearForm(Zh, symmetric=False)
p += grad(ps) * grad(vs) * dxbar
p += (lam * vs + mu * ps) * dxinner
p += (lam * vs + mu * ps) * ds

# R.h.s. term:
pf = LinearForm(Zh)
pf += gfu * grad(vs) * dxbar

pf += p_exact * mu * ds
pf += gfpT * mu * dxinner

#Assembly
p.Assemble()
pf.Assemble()

#Solving the system
gfz.vec.data = p.mat.Inverse(Zh.FreeDofs(),inverse="umfpack") * pf.vec

#For visualization:
gflh.Set(p_exact, definedonelements=ci.GetElementsOfType(HASNEG))
gflam.Set(gfps, definedonelements=ci.GetElementsOfType(HASNEG))

In [ ]:
psl2error = sqrt(Integrate((gfps - p_exact)**2*dX.order(2*order+3), mesh))
psl2error

## Patchwise Post-processing

On each patch, we want to find $p_h^\ast \in \mathbb{P}^{k+1}(\mathcal{T}_\omega)$ so that 
\begin{align}
(\nabla p_h^\ast, \nabla q_h^\ast)_{\Omega \cap \omega} + j_h^\omega(p_h^\ast,q_h^\ast) &= (u_h,\nabla q_h^\ast)_{\Omega \cap \omega} \qquad \forall q_h^\ast \in \mathbb{P}^{k+1}(\mathcal{T}_\omega) \\
(p_h^\ast,1)_{\omega \cap \Omega^{\text{int}}} &= (\bar p_h,1)_{\omega \cap \Omega^{\text{int}}}.
\end{align}

In [ ]:
EA = ElementAggregation(mesh)
EA.Update(ci.GetElementsOfType(NEG), ci.GetElementsOfType(IF))

#Integration domain for ghost penalty
dP = dFacetPatch(definedonelements=EA.patch_interior_facets, deformation=deformation)


#FE Spaces for el-wise post-processing
Vhpbase = L2(mesh, order=order+1, dgjumps=True, dirichlet=[]) # order+1
Lhbase = L2(mesh, order=0, dgjumps=True, dirichlet=[])

Vhp = Restrict(Vhpbase, hasneg)
Lh = Restrict(Lhbase, ci.GetElementsOfType(NEG))
Zh = Vhp * Lh# FESpace([Vhp,Lh],dg_jumps=True)

gflh = GridFunction(Lh)

gfz = GridFunction(Zh)
gfps, gflam = gfz.components

#Test- & Trialfunction
(ps,lam),(vs,mu) = Zh.TnT()

lhs_integrals = [grad(ps) * grad(vs) * dX,
                 (lam * vs + mu * ps) * dxinner,
                 1/h**2 * (ps - ps.Other()) * (vs - vs.Other()) * dP]

rhs_integrals = [gfu * grad(vs) * dX,
                 gfpT * mu * dxinner]

gfz.vec.data = PatchwiseSolve(EA,Zh,sum(lhs_integrals),sum(rhs_integrals),heapsize=900000000)
#For visualization
gflh.Set(p_exact, definedonelements=ci.GetElementsOfType(HASNEG))
gflam.Set(gfps, definedonelements=ci.GetElementsOfType(HASNEG))

psl2error = sqrt(Integrate((gfps - p_exact)**2*dX.order(2*order+3), mesh))

In [ ]:
psl2error